In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [3]:
import os
import gdown
from zipfile import ZipFile
from tqdm import tqdm

file_id = "1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf"
url = f"https://drive.google.com/uc?id={file_id}"

download_path = "./dataset/dacon_dataset.zip"
extract_path = "./dataset/dacon_dataset"

if not os.path.exists(extract_path):
    os.makedirs(extract_path)

gdown.download(url, download_path, quiet=False)

with ZipFile(download_path, 'r') as zip_ref:
    for file in tqdm(zip_ref.namelist(), desc='Extracting files'):
        zip_ref.extract(file, extract_path)

os.remove(download_path)

print(f"File downloaded and extracted to {extract_path}")


Downloading...
From (original): https://drive.google.com/uc?id=1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf
From (redirected): https://drive.google.com/uc?id=1hi1dibkHyFbaxAteLlZJw6r3g9ddd4Lf&confirm=t&uuid=19f496fc-41e8-4478-b6a1-2508f41c8bed
To: /home/hyj/ChanHyung/Audio/DACON_fake_voice_detection/dataset/dacon_dataset.zip
100%|██████████| 3.31G/3.31G [02:07<00:00, 26.0MB/s]
Extracting files: 100%|██████████| 106708/106708 [00:24<00:00, 4304.77it/s]


File downloaded and extracted to ./dataset/dacon_dataset


# Denosing

In [2]:
import glob
import math
import os
import tempfile
import time
from typing import List, Optional, Tuple, Union

import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import torch
from loguru import logger
from PIL import Image
from torch import Tensor

from df import config
from df.enhance import enhance, init_df, load_audio, save_audio
from df.io import resample

from torchaudio import AudioMetaData 

/home/hyj/anaconda3/envs/DACON_DEEPFAKE/lib/python3.11/site-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData


In [3]:
os.getcwd()

'/home/hyj/ChanHyung/Audio/DACON_fake_voice_detection'

In [4]:
import pandas as pd

base_path = os.getcwd()
train_df = pd.read_csv(f'{base_path}/dataset/dacon_dataset/train.csv')
test_df = pd.read_csv(f'{base_path}/dataset/dacon_dataset/test.csv')

In [5]:
from audiomentations import BandStopFilter
augment = BandStopFilter(min_center_freq=100., max_center_freq=8000.0, min_bandwidth_fraction=0.1, max_bandwidth_fraction=0.4, p=1.0)


In [6]:
import torch
from tqdm import tqdm
import shutil

import warnings
warnings.filterwarnings(action='ignore')

dir_name = 'train_bandstop_denoising'

os.makedirs(os.path.join(base_path, 'dataset', 'dacon_dataset', dir_name), exist_ok=True)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, df, _ = init_df(os.path.join(base_path, 'DeepFilterNet2'), config_allow_defaults=True)
model = model.to(device=device).eval()
train_x = os.listdir(os.path.join(base_path, 'dataset', 'dacon_dataset', 'train'))


for i in tqdm(range(len(train_x))):
    id = train_x[i].split('/')[-1].split('.')[0]
    try: 
        path = os.path.join(base_path, 'dataset', 'dacon_dataset', 'train', train_x[i])
        sr = config("sr", 32000, int, section="df")
        sample, meta = load_audio(path, sr)
        sample = augment(sample.numpy(), sample_rate=48000)
        sample = torch.from_numpy(sample)
        enhanced = enhance(model, df, sample)

        lim = torch.linspace(0.0, 1.0, int(sr * 0.15)).unsqueeze(0)
        lim = torch.cat((lim, torch.ones(1, enhanced.shape[1] - lim.shape[1])), dim=1)
        enhanced = enhanced * lim
        if meta.sample_rate != sr:
            enhanced = resample(enhanced, sr, meta.sample_rate)
            sample = resample(sample, sr, meta.sample_rate)
            sr = meta.sample_rate

        noisy_wav = os.path.join(base_path, 'noisy_wav1551.wav')
        save_audio(noisy_wav, sample, sr)
        enhanced_wav = os.path.join(base_path, 'enhanced_wav1551.wav') 
        save_audio(enhanced_wav, enhanced, sr)
        noisy_wav = os.path.join(base_path, 'dataset', 'dacon_dataset', dir_name, f'noisy_{id}.wav')
        shutil.move(os.path.join(base_path, 'noisy_wav1551.wav'), noisy_wav)
        enhanced_wav = os.path.join(base_path, 'dataset', 'dacon_dataset', dir_name, f'enhanced_{id}.wav')
        shutil.move(os.path.join(base_path, 'enhanced_wav1551.wav') ,  enhanced_wav)
    except:
        print(f"너무 짧아서 안됨: {id}")

        

2024-07-23 02:40:31 | INFO     | DF | Running on torch 2.3.0
2024-07-23 02:40:31 | INFO     | DF | Running on host cvlab
2024-07-23 02:40:31 | INFO     | DF | Loading model settings of DeepFilterNet2
2024-07-23 02:40:31 | INFO     | DF | Initializing model `deepfilternet2`


fatal: not a git repository (or any of the parent directories): .git


2024-07-23 02:40:32 | INFO     | DF | Found checkpoint /home/hyj/ChanHyung/Audio/DACON_fake_voice_detection/DeepFilterNet2/checkpoints/model_96.ckpt.best with epoch 96
2024-07-23 02:40:32 | INFO     | DF | Running on device cuda:0
2024-07-23 02:40:32 | INFO     | DF | Model loaded


 19%|█▊        | 10345/55438 [14:33<51:15, 14.66it/s]  

너무 짧아서 안됨: WQNMJRXI


 48%|████▊     | 26770/55438 [38:31<39:36, 12.06it/s]  

너무 짧아서 안됨: ALKRDVFF


100%|██████████| 55438/55438 [1:19:58<00:00, 11.55it/s]


# Speech Segmentation

In [7]:
audio_file_list = [os.path.join(base_path, 'dataset', 'dacon_dataset', 'train', i) 
                   for i in os.listdir(os.path.join(base_path, 'dataset', 'dacon_dataset', 'train'))]

In [8]:
import torchaudio
from speechbrain.inference.VAD import VAD
import os
import json
from tqdm import tqdm

# VAD 모델 초기화
vad_model = VAD.from_hparams(source="speechbrain/vad-crdnn-libriparty", savedir="pretrained_models/vad-crdnn-libriparty")


# 타겟 샘플링 레이트
target_sample_rate = 16000

# 음성 구간 정보를 저장할 디렉토리
output_dir = os.path.join(base_path, 'vad_bandstop_boundaries')
os.makedirs(output_dir, exist_ok=True)

# 오디오 파일 처리
for input_file in tqdm(audio_file_list):
    # Step 1: 오디오 파일 로드 및 샘플링 레이트 변환
    signal, original_sample_rate = torchaudio.load(input_file)
    transform = torchaudio.transforms.Resample(orig_freq=original_sample_rate, new_freq=target_sample_rate)
    resampled_signal = transform(signal)
    
    # 오디오 길이 계산 (초 단위)
    audio_length = resampled_signal.size(1) / target_sample_rate
    
    # 임시 파일로 저장
    temp_output_file = os.path.join(base_path, 'temp_resampled_audio_file2.wav')
    torchaudio.save(temp_output_file, resampled_signal, target_sample_rate)
    
    # Step 2: 변환된 파일로 VAD 수행
    boundaries = vad_model.get_speech_segments(temp_output_file)
    
    # Step 3: 각 음성 구간의 시작 시간과 종료 시간을 기록
    base_filename = os.path.splitext(os.path.basename(input_file))[0]
    boundaries_list = []
    for boundary in boundaries:
        start, end = boundary[0].item(), boundary[1].item()
        boundaries_list.append({
            "start": start,
            "end": end
        })
    
    # JSON 파일로 저장
    output_data = {
        "audio_length": audio_length,
        "boundaries": boundaries_list
    }
    boundaries_filename = f"{base_filename}_boundaries.json"
    boundaries_filepath = os.path.join(output_dir, boundaries_filename)
    with open(boundaries_filepath, 'w') as f:
        json.dump(output_data, f)
    
    # print(f"Saved boundaries and audio length for {input_file} to {boundaries_filepath}")
    
    # 임시 파일 삭제
    os.remove(temp_output_file)

print("All boundaries and audio lengths have been saved.")


torchvision is not available - cannot save figures


100%|██████████| 55436/55436 [5:33:52<00:00,  2.77it/s]  

All boundaries and audio lengths have been saved.


# Make (0, 0)

In [10]:
import os
import json
import librosa
import numpy as np
import soundfile as sf
from tqdm import tqdm

# 경로 설정
json_dir = os.path.join(base_path, 'vad_bandstop_boundaries')
audio_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_bandstop_denoising')
output_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_silent')


# 출력 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

# 스킵할 이름 리스트
skip_names = [
    "XVYZXEIZ", "ODYZCZMF", "EHKFWHIN", "LSDLYESX", "KNBWYZNI",
    "ILYTDFAJ", "ENXDYYVM", "TTDERPVZ", "LPSIJBXB", "CHSDQDKV",
    "MRNJWEFU", "EBJXOVTJ", "MXDGGMDP", "VSLMVONT", "XFZFVHLE",
    "OGZHVEUS", "FVFYPRZP", "WQNMJRXI", "ALKRDVFF"
]

# JSON 파일 목록 가져오기
json_files = [f for f in os.listdir(json_dir) if f.endswith('_boundaries.json')]

# JSON 파일 처리
for json_file in tqdm(json_files, desc="Processing JSON files"):
    name = json_file.replace('_boundaries.json', '')
    
    # 스킵할 이름인지 확인
    if name in skip_names:
        continue
    
    # JSON 파일 경로
    json_path = os.path.join(json_dir, json_file)
    
    # 오디오 파일 경로
    audio_path = os.path.join(audio_dir, f"{name}.wav")
    
    # JSON 파일 읽기
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # 오디오 파일 읽기
    audio_data, sr = librosa.load(audio_path, sr=None)
    
    # boundaries 가져오기
    boundaries = data.get('boundaries', [])
    
    # 간격 설정 (여기서는 0.1초를 추가로 잘라냅니다)
    margin = 0.1  # 100 milliseconds
    
    if not boundaries:
        # boundaries가 없으면 전체 오디오 데이터 사용
        cropped_audio = audio_data
    else:
        # 제외할 구간 설정
        exclude_intervals = [(max(boundary['start'] - margin, 0), min(boundary['end'] + margin, len(audio_data) / sr)) for boundary in boundaries]
        
        # 제외할 구간을 제외한 오디오 데이터 생성
        keep_samples = []
        prev_end = 0.0
        for start, end in exclude_intervals:
            if prev_end < start:
                keep_samples.append(audio_data[int(prev_end * sr):int(start * sr)])
            prev_end = end
        if prev_end * sr < len(audio_data):
            keep_samples.append(audio_data[int(prev_end * sr):])
        
        # 모든 유지할 샘플들을 합침
        cropped_audio = np.concatenate(keep_samples) if keep_samples else np.array([], dtype=audio_data.dtype)
    
    # 결과 오디오 파일 저장
    output_audio_path = os.path.join(output_dir, f"cropped_{name}.wav")
    sf.write(output_audio_path, cropped_audio, sr)


Processing JSON files: 100%|██████████| 55436/55436 [01:29<00:00, 619.31it/s]


In [11]:
import os
import soundfile as sf
import numpy as np
from tqdm import tqdm

# 경로 설정
output_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_silent')
merged_output_path = os.path.join(base_path, 'merged_audio.wav')


# 병합할 오디오 파일 목록 가져오기
audio_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.wav')]

# 병합할 오디오 데이터를 저장할 리스트 초기화
merged_audio_data = []
sample_rate = None

# 각 오디오 파일을 읽고 병합
for audio_file in tqdm(audio_files, desc="Merging audio files"):
    audio_data, sr = sf.read(audio_file)
    if sample_rate is None:
        sample_rate = sr  # 첫 번째 오디오 파일의 샘플링 레이트 사용
    else:
        assert sample_rate == sr, f"Sample rate mismatch: {audio_file}"
    merged_audio_data.append(audio_data)

# 병합된 오디오 데이터를 하나의 배열로 결합
if merged_audio_data:
    merged_audio_data = np.concatenate(merged_audio_data)
    # 병합된 오디오 데이터를 새로운 파일로 저장
    sf.write(merged_output_path, merged_audio_data, sample_rate)
    print(f"Merged audio file saved at {merged_output_path}")
else:
    print("No audio files to merge.")


Merging audio files: 100%|██████████| 55436/55436 [00:05<00:00, 10755.31it/s]


Merged audio file saved at /home/hyj/ChanHyung/Audio/DACON_fake_voice_detection/merged_audio.wav


# (1, 0), (0, 1) Analysis

# save train segment

In [12]:
import pandas as pd
from pydub import AudioSegment
import json
from tqdm import tqdm



In [15]:
import soundfile as sf
import json

output_base_path = os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_segment')

os.makedirs(output_base_path, exist_ok=True)

results = []

for _, row in tqdm(train_df.iterrows(), total=train_df.shape[0]):
    json_path = os.path.join(base_path, 'vad_bandstop_boundaries', f"enhanced_{row['id']}_boundaries.json")
    

    if row['id'] in ['WQNMJRXI', 'ALKRDVFF']:
        continue

    audio_path = os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_bandstop_denoising', f'enhanced_{row["id"]}.wav')
    
    # 오디오 파일 로드
    data, samplerate = sf.read(audio_path)
    
    # JSON 파일에서 boundaries 읽기
    with open(json_path, 'r') as file:
        data_json = json.load(file)
        for i, boundary in enumerate(data_json['boundaries']):
            start_sample = int(boundary['start'] * samplerate)  # 초를 샘플 인덱스로 변환
            end_sample = int(boundary['end'] * samplerate)  # 초를 샘플 인덱스로 변환
            
            # 잘라낸 오디오 부분 추출
            cut_data = data[start_sample:end_sample]
            
            # 잘라낸 오디오 파일 저장
            output_path = f"{output_base_path}/{row['id']}_part{i}.wav"
            sf.write(output_path, cut_data, samplerate)

            results.append({
                'id': row['id'],
                'path': output_path,
                'label': row['label']
            })


100%|██████████| 55438/55438 [02:42<00:00, 341.98it/s]


In [16]:
results = pd.DataFrame(results)

# Make (1, 0), (0, 1)

In [24]:
real_df = results[results['label'] == 'real']
real_df

,id,path,label
0,RUNQPNJF,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,real
2,RDKEKEVX,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,real
3,QYHJDOFK,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,real
4,RSPQNHAO,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,real
6,ICDBLWYI,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,real
...,...,...,...
55477,KLDKERGF,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,real
55478,JTGPEAPT,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,real
55480,QULDVNGC,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,real
55481,QAYTRNGK,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,real


In [25]:
output_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_bandstop_new_dataset')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

file_index = 0
datasets = []


for index, row in tqdm(real_df.iterrows(), total=real_df.shape[0]):
    file_path = row['path']
    data, samplerate = sf.read(file_path)

    # 5초(5000 밀리초) 단위로 분할
    chunk_length_ms = 5000  # 분할 길이 (5초)
    chunk_length_samples = int(samplerate * (chunk_length_ms / 1000))

    # 각 조각 저장
    for i in range(0, len(data), chunk_length_samples):
        chunk = data[i:i + chunk_length_samples]
        if len(chunk) >= int(samplerate * 0.44):  # 0.44초 이상인 경우에만 저장
            output_path = os.path.join(output_dir, f"TRAIN_{file_index:05d}.wav")
            sf.write(output_path, chunk, samplerate, format='WAV')
            file_index += 1

            datasets.append({
                'id1': row['id'],
                'id2': None,
                'path': output_path,
                'fake': 0,
                'real': 1,
            })

100%|██████████| 27631/27631 [01:45<00:00, 260.91it/s]


In [26]:
fake_df = results[results['label'] == 'fake']

In [27]:
output_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_bandstop_new_dataset')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for index, row in tqdm(fake_df.iterrows(), total=fake_df.shape[0]):
    file_path = row['path']
    data, samplerate = sf.read(file_path)

    # 5초(5000 밀리초) 단위로 분할
    chunk_length_ms = 5000  # 분할 길이 (5초)
    chunk_length_samples = int(samplerate * (chunk_length_ms / 1000))

    # 각 조각 저장
    for i in range(0, len(data), chunk_length_samples):
        chunk = data[i:i + chunk_length_samples]
        if len(chunk) >= int(samplerate * 0.44):  # 0.44초 이상인 경우에만 저장
            output_path = os.path.join(output_dir, f"TRAIN_{file_index:05d}.wav")
            sf.write(output_path, chunk, samplerate, format='WAV')
            file_index += 1

            datasets.append({
                'id1': row['id'],
                'id2': None,
                'path': output_path,
                'fake': 1,
                'real': 0,
            })

100%|██████████| 27856/27856 [00:56<00:00, 495.71it/s]


# JSON analysis

In [28]:
train_df_no_label = pd.DataFrame(datasets)

In [29]:
train_df_no_label

,id1,id2,path,fake,real
0,RUNQPNJF,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
1,RDKEKEVX,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
2,RDKEKEVX,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
3,QYHJDOFK,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
4,RSPQNHAO,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
...,...,...,...,...,...
63590,XVFCHHQZ,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,0
63591,SZXIACUZ,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,0
63592,PXLBTGRH,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,0
63593,CGGQGPOQ,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,0


In [30]:
import pandas as pd

# fake, real 비율 계산
fake_ratio = train_df_no_label['fake'].mean()
real_ratio = train_df_no_label['real'].mean()

# 결과 출력
print(f"Fake Ratio: {fake_ratio:.2%}")
print(f"Real Ratio: {real_ratio:.2%}")


Fake Ratio: 43.71%
Real Ratio: 56.29%


In [31]:
train_df_no_label['fake'].sum(), train_df_no_label['real'].sum()

(27798, 35797)

In [33]:
result_df = train_df_no_label
result_df

,id1,id2,path,fake,real
0,RUNQPNJF,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
1,RDKEKEVX,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
2,RDKEKEVX,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
3,QYHJDOFK,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
4,RSPQNHAO,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
...,...,...,...,...,...
63590,XVFCHHQZ,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,0
63591,SZXIACUZ,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,0
63592,PXLBTGRH,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,0
63593,CGGQGPOQ,None,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,0


# Make Fake * 2, Real * 2 Fake and Real 

In [35]:
import random
from pydub import AudioSegment
import pandas as pd
from tqdm import tqdm
import os

def combine_audios(audio1_path, audio2_path, fake, real, i):
    try:
        # Load the audio files
        audio1 = AudioSegment.from_file(audio1_path)
        audio2 = AudioSegment.from_file(audio2_path)
        
        # Ensure both audios are shorter than 5 seconds
        audio1 = audio1.set_frame_rate(32000)[:5000]
        audio2 = audio2.set_frame_rate(32000)[:5000]

        # Determine the lengths of the audio files in samples
        len_audio1 = len(audio1)
        len_audio2 = len(audio2)

        # Determine random start positions
        start1 = random.randint(0, max(0, 5000 - len_audio1))
        start2 = random.randint(0, max(0, 5000 - len_audio2))

        # Create a silent audio segment of 5 seconds
        combined = AudioSegment.silent(duration=5000, frame_rate=32000)

        # Overlay the audios at the random start positions
        combined = combined.overlay(audio1, position=start1)
        combined = combined.overlay(audio2, position=start2)
        
        # Define the output path
        
        new_path = os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_bandstop_new_dataset', f"TRAIN_{i:05d}.wav")

        # Ensure the directory exists
        os.makedirs(os.path.dirname(new_path), exist_ok=True)
        
        # Export the combined audio
        combined.export(new_path, format="wav")
        
        # Return metadata for the combined audio
        return {
            'id1': audio1_path,
            'id2': audio2_path,
            'path': new_path,
            'fake': fake,
            'real': real
        }
    except Exception as e:
        print(f"Error processing files {audio1_path} and {audio2_path}: {e}")
        return None

def create_combinations(df1, df2, fake, real, target_count, start_index):
    new_results = []
    for i in tqdm(range(start_index, start_index + target_count)):
        # Ensure different samples are picked from each dataframe
        audio1_path = random.choice(list(df1['path']))
        audio2_path = random.choice(list(df2['path']))
        while audio1_path == audio2_path:
            audio1_path = random.choice(list(df1['path']))
            audio2_path = random.choice(list(df2['path']))
        result = combine_audios(audio1_path, audio2_path, fake, real, i)
        if result:
            new_results.append(result)
    return new_results

# Assuming train_new_dataset.csv contains paths to audio files and a column 'type' indicating 'fake' or 'real'
df = result_df.copy()

fake_df = df[df['fake'] == 1]
real_df = df[df['real'] == 1]

# Example usage:
start_index = len(result_df)
new_results = []

(43977, 35875)

new_results.extend(create_combinations(fake_df, fake_df, fake=2, real=0, target_count=60000-result_df['fake'].sum(), start_index=start_index))
start_index += 60000-result_df['fake'].sum()
new_results.extend(create_combinations(real_df, real_df, fake=0, real=2, target_count=60000-result_df['real'].sum(), start_index=start_index))
start_index += 60000-result_df['real'].sum()
new_results.extend(create_combinations(fake_df, real_df, fake=1, real=1, target_count=60000, start_index=start_index))


  0%|          | 32/32202 [00:00<03:21, 159.51it/s]

100%|██████████| 60000/60000 [04:51<00:00, 205.88it/s]


In [36]:
new_results = pd.DataFrame(new_results)
new_results

,id1,id2,path,fake,real
0,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,2,0
1,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,2,0
2,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,2,0
3,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,2,0
4,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,2,0
...,...,...,...,...,...
116400,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,1
116401,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,1
116402,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,1
116403,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,1


In [37]:
combined_df = pd.concat([result_df, new_results], ignore_index=True)
combined_df.to_csv(os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_bandstop_new_dataset_addmix.csv'), index=False)
 

# Make 0 0

In [38]:
combined_df = pd.read_csv(os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_bandstop_new_dataset_addmix.csv'))
combined_df

,id1,id2,path,fake,real
0,RUNQPNJF,NaN,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
1,RDKEKEVX,NaN,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
2,RDKEKEVX,NaN,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
3,QYHJDOFK,NaN,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
4,RSPQNHAO,NaN,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,1
...,...,...,...,...,...
179995,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,1
179996,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,1
179997,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,1
179998,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,1,1


In [39]:
zero_zero_dats = []

In [40]:
import os
import librosa
import soundfile as sf
from tqdm import tqdm

# 경로 설정
audio_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_bandstop_denoising')
output_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_bandstop_new_dataset')


# 파일 이름 리스트
file_names = [
    'GPBVYPQM', 'ZMHQQOIX', 'TSDWJRYK', 'KHETVYDC', 'UCFUFGVP',
    'BAUSKCYH', 'NPYSZXLN', 'XWQKSBNA', 'CTBCEJWY', 'IZROHJFG',
    'MMWOXBSY', 'NSXAPHSR', 'BTVLPBVC', 'VJPOKGUR', 'SJYXSIFH',
    'NHKKJSSF', 'FYQBNCIR', 'AXHPTWRL', 'JBIYWASW', 'UJKRSIJO',
    'SANNRLIG', 'QLHQKYHB', 'AZRNUMHQ', 'TJGJDQSC', 'SULPUEVQ',
    'YGLREVCI', 'ETJAABZE'
]

i = 150000

# 각 파일 처리
for name in tqdm(file_names, desc="Processing audio files"):
    # 오디오 파일 경로
    audio_path = os.path.join(audio_dir, f"enhanced_{name}.wav")
    
    # 오디오 파일 읽기
    audio_data, sr = librosa.load(audio_path, sr=None)
    
    # 총 길이 (초)
    total_duration = librosa.get_duration(y=audio_data, sr=sr)
    
    # 5초씩 자르기
    for start in range(0, int(total_duration), 5):
        end = start + 5
        if end <= total_duration:
            segment = audio_data[int(start * sr):int(end * sr)]
            output_audio_path = os.path.join(output_dir, f"TRAIN_{i:05d}.wav")
            sf.write(output_audio_path, segment, sr)
            i += 1
            zero_zero_dats.append({
                    'id1': audio_path,
                    'id2': "",
                    'path': output_audio_path,
                    'fake': 0,
                    'real': 0
                })
            
        else:
            # 5초 미만의 segment는 저장하지 않음
            break


Processing audio files: 100%|██████████| 27/27 [00:00<00:00, 230.35it/s]


In [41]:
# 이 부분은 skip
# 00:19-00:20
# 00:30-00:37
# 14:23-14:25
# 28:12-28:13
# 29:00-29:04

import os
import librosa
import soundfile as sf
import numpy as np
from tqdm import tqdm

# 경로 설정
input_audio_path = os.path.join(base_path, 'merged_audio.wav')
output_dir = os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_bandstop_new_dataset')

# 출력 디렉토리 생성
os.makedirs(output_dir, exist_ok=True)

# 제외할 구간 (초 단위)
exclude_intervals = [
    (19, 20),
    (30, 37),
    (14*60+23, 14*60+25),
    (28*60+12, 28*60+13),
    (29*60, 29*60+4)
]

i = 150020

# 오디오 파일 읽기
audio_data, sr = librosa.load(input_audio_path, sr=None)

# 총 길이 (초)
total_duration = librosa.get_duration(y=audio_data, sr=sr)

# 제외할 구간을 제외한 오디오 데이터 생성
keep_samples = []
prev_end = 0.0
for start, end in sorted(exclude_intervals):
    if prev_end < start:
        keep_samples.append(audio_data[int(prev_end * sr):int(start * sr)])
    prev_end = end
if prev_end * sr < len(audio_data):
    keep_samples.append(audio_data[int(prev_end * sr):])

# 모든 유지할 샘플들을 합침
processed_audio = np.concatenate(keep_samples) if keep_samples else np.array([], dtype=audio_data.dtype)

# 5초씩 자르기
start = 0
segment_count = 0
for start in tqdm(range(0, len(processed_audio), 5 * sr), desc="Processing audio segments"):
    end = start + 5 * sr
    if end <= len(processed_audio):
        segment = processed_audio[start:end]
        output_audio_path = os.path.join(output_dir, f"TRAIN_{i:05d}.wav")
        sf.write(output_audio_path, segment, sr)
        segment_count += 1

        i += 1
        zero_zero_dats.append({
                'id1': audio_path,
                'id2': "",
                'path': output_audio_path,
                'fake': 0,
                'real': 0
            })
    else:
        # 마지막 segment가 5초 미만이면 저장하지 않음
        break

print("Processing completed.")


Processing audio segments: 100%|█████████▉| 431/432 [00:01<00:00, 318.52it/s]

Processing completed.


In [42]:
data_paths = [i['path'] for i in zero_zero_dats]
len(data_paths)

451

In [43]:
result_df = pd.DataFrame(zero_zero_dats)

In [44]:
combined_df = pd.concat([combined_df, result_df], ignore_index=True)
combined_df.to_csv(os.path.join(base_path, 'dataset', 'dacon_dataset', 'train_bandstop_new_dataset_addmix.csv'), index=False)


In [45]:
combined_df.tail()

,id1,id2,path,fake,real
180446,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,0
180447,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,0
180448,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,0
180449,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,0
180450,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,,/home/hyj/ChanHyung/Audio/DACON_fake_voice_det...,0,0
